# **About this dataset**
### Naming convention for files -> C_NNNNN_BB_SS_OOOOO_RXX, where:

* C = emission context (values: B = brushing; F = waiting for food; I: isolation in an unfamiliar environment);
* NNNNN = cat’s unique ID;
* BB = breed (values: MC = Maine Coon; EU: European Shorthair);
* SS = sex (values: FI = female, intact; FN: female, neutered; MI: male, intact; MN: male, neutered);
* OOOOO = cat owner’s unique ID;
* R = recording session (values: 1, 2 or 3)
* XX = vocalization counter (values: 01..99)ter (values: 01..99)
 01..99)

In [6]:
import os
import IPython.display as ipd

import librosa
import librosa.display
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import utils
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.callbacks import Callback
from time import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns

from scipy.io import wavfile #read and write soundfile (.wav - time series)
from IPython.display import Audio #play file sound in colab

import random

# Import Dataset

In [7]:
import os

DATA_DIR = r'C:\Users\nicolette\Desktop\CatMe\dataset'  # <- adjust to your folder
audio_files = os.listdir(DATA_DIR)
print(audio_files[:5])  # print first 5 files


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\nicolette\\Desktop\\CatMe\\dataset'

In [ ]:
# 'F' : 'Waiting For Food', 'I' : 'Isolated in unfamiliar Environment', 'B' : 'Brushing'
emission_context = {'F' : [], 'I' : [], 'B' : []}

# breed (values: MC = Maine Coon; EU: European Shorthair);
breed = {'MC' : [], 'EU' : []}

for file in audio_files:
    split = file.split('_')
    if split[0] in emission_context.keys():
        emission_context.get(split[0]).append(file)
        
    if split[2] in breed.keys():
        breed.get(split[2]).append(file)

-----

# Data Understanding

### Waiting for food

In [ ]:
wait_food_ex = '/kaggle/input/cat-meow-classification/dataset/dataset/'+ emission_context.get('F')[2]
data , sample_rate = librosa.load(wait_food_ex)
librosa.load(wait_food_ex)
ipd.Audio(wait_food_ex)

In [ ]:
# Load the audio file using librosa
data, sample_rate = librosa.load(wait_food_ex, sr=None)

# Calculate time values for the x-axis
time = librosa.times_like(data, sr=sample_rate)

# Plot the amplitude vs. time graph
plt.figure(figsize=(12, 6))
plt.plot(time, data, label="Waveform")
plt.title("Example Sound of Waiting for food")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.legend()
plt.grid()
plt.show()

### Brushing (being brushed affectionately by the owner)

In [ ]:
brushing_ex = '/kaggle/input/cat-meow-classification/dataset/dataset/'+ emission_context.get('B')[2]
data , sample_rate = librosa.load(brushing_ex)
librosa.load(brushing_ex)
ipd.Audio(brushing_ex)

In [ ]:
# Load the audio file using librosa
data, sample_rate = librosa.load(brushing_ex, sr=None)

# Calculate time values for the x-axis
time = librosa.times_like(data, sr=sample_rate)

# Plot the amplitude vs. time graph
plt.figure(figsize=(12, 6))
plt.plot(time, data, label="Waveform")
plt.title("Example Sound of Brushing")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.legend()
plt.grid()
plt.show()

### Isolation in unfamiliar environment

In [5]:
isolate_ex = '/kaggle/input/cat-meow-classification/dataset/dataset/'+ emission_context.get('I')[2]
data , sample_rate = librosa.load(isolate_ex)
librosa.load(isolate_ex)
ipd.Audio(isolate_ex)

NameError: name 'emission_context' is not defined

In [ ]:
# Load the audio file using librosa
data, sample_rate = librosa.load(isolate_ex, sr=None)

# Calculate time values for the x-axis
time = librosa.times_like(data, sr=sample_rate)

# Plot the amplitude vs. time graph
plt.figure(figsize=(12, 6))
plt.plot(time, data, label="Waveform")
plt.title("Example Sound of Isolate")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.legend()
plt.grid()
plt.show()

-----------

# Data preparation

In [ ]:
def cleanse_audio(file_path, target_sample_rate=16000):
    """
    Cleanses the audio file by performing the following:
    - Resampling to the target sample rate
    - Normalization
    - Trimming silence
    """
    # Load audio file
    data, sr = librosa.load(file_path, sr=None)

    # Trim silence from the beginning and end
    data, _ = librosa.effects.trim(data)

    # Resample to the target sample rate
    if sr != target_sample_rate:
        data = librosa.resample(data, orig_sr=sr, target_sr=target_sample_rate)
        sr = target_sample_rate

    # Normalize the audio to range [-1, 1]
    data = librosa.util.normalize(data)

    return data, sr

In [ ]:
# Feature Extraction
# extract these features from audio files.

# def extract_features(file_path):
#     y, sr = librosa.load(file_path, duration=5.0)  # Load the audio file (5 sec duration)
#     mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
#     chroma = librosa.feature.chroma_stft(y=y, sr=sr)
#     mel = librosa.feature.melspectrogram(y=y, sr=sr)
#     contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    
#     features = np.hstack([
#         np.mean(mfcc, axis=1),
#         np.mean(chroma, axis=1),
#         np.mean(mel, axis=1),
#         np.mean(contrast, axis=1)
#     ])
    # return features

def extract_features(file_path, target_sample_rate=16000):
    """
    Cleanses the audio and extracts features (MFCCs, Chroma, Spectral Contrast).
    """
    # Cleanse the audio file
    data, sr = cleanse_audio(file_path, target_sample_rate=target_sample_rate)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=13).mean(axis=1)

    # Extract Chroma features
    chroma = librosa.feature.chroma_stft(y=data, sr=sr).mean(axis=1)

    # Extract Spectral Contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=data, sr=sr).mean(axis=1)

    # Combine all features
    return np.hstack([mfccs, chroma, spectral_contrast])


In [ ]:
# Preprocessing data

DATA_DIR = '../input/cat-meow-classification/dataset/dataset/'
audio_files = os.listdir(DATA_DIR)

# Prepare labels
emission_context = {'F': 'Waiting For Food', 'I': 'Isolated in unfamiliar Environment', 'B': 'Brushing'}
breed_map = {'MC': 'Maine Coon', 'EU': 'European Shorthair'}

data = []

for file in audio_files:
    split = file.split('_')
    label_context = split[0]
    label_breed = split[2]
    
    file_path = os.path.join(DATA_DIR, file)
    features = extract_features(file_path)
    data.append([features, label_context, label_breed])

# Convert to DataFrame
df = pd.DataFrame(data, columns=['features', 'emission_context', 'breed'])


In [ ]:
# Train test spilt

from sklearn.model_selection import train_test_split

X = np.array(df['features'].tolist())
y_context = df['emission_context']
y_breed = df['breed']

# Split for both classification tasks
X_train, X_test, y_train_context, y_test_context = train_test_split(X, y_context, test_size=0.2, random_state=42)
_, _, y_train_breed, y_test_breed = train_test_split(X, y_breed, test_size=0.2, random_state=42)

# Data modeling

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train context classifier
context_clf = RandomForestClassifier(random_state=42)
context_clf.fit(X_train, y_train_context)

# Evaluation

In [ ]:
from sklearn.metrics import classification_report

# Evaluate context classifier
y_pred_context = context_clf.predict(X_test)
print("Emission Context Classification Report:")
print(classification_report(y_test_context, y_pred_context))
